In [13]:
library(tidyverse)
library(repr)
library(tidymodels)

In [14]:
# reading the dataframe from the document 

heart_data <- read_delim("data/processed.cleveland.data", delim=",", col_names = FALSE)

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [15]:
# assigning coloumn names to each column

heart_data <- rename(heart_data,
                    age = X1,
                    sex = X2,
                    cp = X3,
                    trestbps = X4,
                    chol = X5,
                    fbs = X6,
                    restecg = X7,
                    thalach = X8,
                    exang = X9,
                    oldpeak = X10,
                    slope = X11,
                    ca = X12,
                    thal = X13,
                    num = X14)


In [16]:
# removing "?" from data and replacing it with NA

heart_data[heart_data == "?"] <- NA


In [18]:
# selecting columns of interest: age, sex, chol and num

heart_data <- select(heart_data, age, sex, chol, num)

In [21]:
heart_data_clean <- heart_data |>
    mutate(age = as_factor(age)) |>
    mutate(sex = as_factor(sex)) |>
    mutate(sex = fct_recode(sex, "M" = "1", "F" = "0")) |>
    mutate(chol = as_factor(chol)) |>
    mutate(num = as_factor(num)) |>
    mutate(num = fct_recode(num, "H" = "0", "D" = "1", "D" = "2", "D" = "3", "D" = "4"))

In [24]:
# splitting dataframe into training and testing datasets

heart_split <- initial_split(heart_data_clean, prop = 0.75, strata = num)
heart_training <- training(heart_split)
heart_testing <- testing(heart_split)

In [ ]:
# getting the min, max, and mean of each predictor



In [20]:
num_obs <- nrow(heart_data_clean)
percent_obs <- heart_data_clean |> 
    group_by(num) |>
    summarize(
        count = n(),
        percentage = n()/num_obs * 100)
percent_obs

num,count,percentage
<fct>,<int>,<dbl>
H,164,54.12541
D,139,45.87459
